In [ ]:
import pandas as pd
import pickle as pickle 
with open('data_with_coord_29_04_21', "rb") as fh:
    df = pickle.load(fh)
df = df[df.dist_euclidian<150000]
df.info()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

subset = df.sample(50000)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(subset.dist_euclidian, subset.AttendanceTimeSeconds, '.', markersize=3)
ax.set_xlabel('Distance euclidienne (m)', fontsize=20)
ax.set_ylabel('Temps (s)', fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='both', which='major', labelsize=15)
plt.tight_layout()
fig.show()

In [ ]:
subset['velocity'] = subset.dist_euclidian/subset.AttendanceTimeSeconds
print('la vélocité minimale est de ', subset.velocity.min(),'m/s, et maximale est de ', subset.velocity.max(), 'ms')
subset['kmh'] = (subset.velocity*3600)/1000
print('Ce qui correspond à une vitesse de ', subset.kmh.min(),'Km/H, et maximale est de ', subset.kmh.max(), 'Km/H')

# On peut donc constater que dans les données doivent encore se trouver quelques anomalies :
- Des interventions très proches de la station, mais pour lesquelles les pompiers ont pris plusieurs minutes,
- Des interventions lointaines, mais pour lesquelles les pompiers ont pu recourir à des véhicules plus rapides (Hélicoptères, formule 1, téléporteur, ou encore Sleipnir, fils de Loki, Destrier d'Odin, le cheval à 8 jambes capable de survoler les mers)
- Des interventions différées (relèves, excercices, effort prolongé de lutte contre le feu...)

### Comme nous souhaitons nous focaliser sur les évenements necessitant une intervention classique (véhicules), on peut appliquer quelques filtres supplémentaires :
- Temps d'intervention > 0s
- Distance euclidienne > 100m
- Année > 2016 (cf analyse temps de réponse du rapport)
- velocity < 50 (180 km/h) & velocity > 7 (25 km/h) 

In [ ]:
subset = df.copy()
subset = subset[(df.AttendanceTimeSeconds >0) & (subset.dist_euclidian>100)]
subset['velocity'] = subset.dist_euclidian / subset.AttendanceTimeSeconds
subset = subset[(subset.CalYear>2016) & (subset.velocity <50) & (subset.velocity > 7)]
print("la base de données est donc réduite de :", round(100-(subset.shape[0]/df.shape[0])*100, 2), '% et compte maintenant ', subset.shape[0],' entrées')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(subset.dist_euclidian, subset.AttendanceTimeSeconds, '.', markersize=1)
ax.set_xlabel('Distance euclidienne (m)', fontsize=20)
ax.set_ylabel('Temps (s)', fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='both', which='major', labelsize=15)
plt.tight_layout()
fig.show()

# La distribution des données ne semble pas s'offrir à l'utilisation d'une régression linéaire. 
### Nous suggérons qu'elles dépendent de trop de facteurs différents pour offrir une prédiction satisfaisante sans davantage de détails logistiques (absent de la base de données)
### Nous avons alors imaginé de produire des informations pertinentes et descripives, en fonction d'un lieu d'intervention.
### Il nous faut donc déterminer pour chaque point de la carte, la station qui devrait intervenir

# Si on observe les points d'interventions par station :

In [ ]:
import seaborn as sns
import numpy as np
cmap=plt.cm.get_cmap(plt.cm.rainbow)
max_s = len(subset.DeployedFromStation_Name.unique())
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
for i,s in enumerate(subset.DeployedFromStation_Name.unique()):
    ax.plot(subset[subset.DeployedFromStation_Name==s].x_utm, 
            subset[subset.DeployedFromStation_Name==s].y_utm,
           '.',
           color=cmap(i/max_s),
           markersize=1)
#sns.scatterplot(x='x_utm', y = 'y_utm', hue='DeployedFromStation_Name',legend=False,data=subset, sizes=0.2, ax=ax);
ax.set_xlabel('X', fontsize=20)
ax.set_ylabel('Y', fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(labelsize=15)
plt.tight_layout()
fig.show()

### Les données semblent difficiles à distinguer, et un kmeans appliqué directement risque de générer beaucoup d'erreurs
### Nous avons eu l'idée de renforcer l'apprentissage en augmentant la distinctivité entre les stations
### C'est à dire, de classer des points intermédiaire sur la distance station-intervention, plutôt que la destination elle même
Exemple lorsqu'on prend la moitié de cette distance:

In [ ]:
subset['middle_x'] = subset.o_x_utm + (1/2)*(subset.x_utm-subset.o_x_utm)
subset['middle_y'] = subset.o_y_utm + (1/2)*(subset.y_utm-subset.o_y_utm)

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
for i,s in enumerate(subset.DeployedFromStation_Name.unique()):
    ax.plot(subset[subset.DeployedFromStation_Name==s].middle_x, 
            subset[subset.DeployedFromStation_Name==s].middle_y,
           '.',
           color=cmap(i/max_s),
           markersize=1)
ax.set_xlabel('X', fontsize=20)
ax.set_ylabel('Y', fontsize=20)
ax.tick_params(labelsize=15)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
fig.show()

### Les points semblent plus propices à la classification par kmeans.
### On peut donc généraliser l'utilisation des distances pour trouver un ratio optimal (qui maximise la classification):

In [ ]:
centers = []
for i in subset.DeployedFromStation_Name.unique():
    # pour chaque station, on récupère les coordonnées x y, ainsi que le nom
    x, y = subset[subset.DeployedFromStation_Name==i].iloc[0, :].o_x_utm, subset[subset.DeployedFromStation_Name==i].iloc[0, :].o_y_utm
    centers.append((x, y, i))
centers = np.array(centers)
from sklearn.cluster import KMeans

# sample car les données sont nombreuses et c'est exagéré pour le modèle
# subset = subset.sample(50000)
def learn_test(a, b, subset):
    # le modèle marche mieux avec des morceau de segment, contrôle de la portion la plus performante à faire
    subset['middle_x'] = subset.o_x_utm + (a/b)*(subset.x_utm-subset.o_x_utm)
    subset['middle_y'] = subset.o_y_utm + (a/b)*(subset.y_utm-subset.o_y_utm)
    
    index = np.arange(subset.shape[0])
    subset = subset.set_index(index)
    np.random.shuffle(index)
    ratio_test = int(0.2*subset.shape[0])
    # on calcul le poids de chaque station (nombre d'intervention par station / nombre total d'intervention)
    w = np.array(subset.loc[index[:-ratio_test], ['DeployedFromStation_Name']].value_counts().tolist()) / (subset.shape[0]-ratio_test)

    # on initialise la mixture gaussienne avec autant de centre que de station, initialisé sur les stations,
    gm = KMeans(n_clusters=centers.shape[0], init=centers[:, :2], n_init=1, random_state=1337)

    # fit et sub pour le test
    gm.fit(subset.loc[index[:-ratio_test], ['middle_x','middle_y']])
    print(a, b, " fit ok")
    y = subset.loc[index[-ratio_test:], ['DeployedFromStation_Name']]
    
    y_pred = gm.predict(subset.loc[index[-ratio_test:], ['x_utm','y_utm']])

    print(a, b, " collecte ok")
    success = 0
    predicted = centers[y_pred, -1]
    success = np.where(y.DeployedFromStation_Name==predicted)[0].shape[0]/y.shape[0]
    return success

results_km = []
for a in range(11):
    results_km.append(learn_test(a, 10, subset))
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(results_km)
ax.set_xticks(np.arange(11))
ax.set_xticklabels((np.arange(11)*0.1).round(2))
ax.set_xlabel('Ratio du segment station-incident', fontsize=25)
ax.set_ylabel('Ratio de réussite', fontsize=25)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
plt.show()

# La performance moyenne avec la méthode Kmeans n'est pas fantastique (max 63%). Nous pensons que la cause principale de ce score est le fait que les stations ont fréquemment pu intervenir sur des secteurs communs

### Une solution pertinente serait, pour chaque lieu d'incident, de déterminer quels sont les stations qui sont susceptibles d'y intervenir. 
Bien sûr, on pourrait simplement utiliser la distance, mais il serait difficile de fixer le seuil : un incident en campagne pourrait faire intervenir 2-3 stations situées à 10-15 km, mais un incident en ville pourrait faire intervenir 4-5 stations situées à 3 km. Gérer ces spécificités individuellement pourrait s'avérer plus compliqué qu'appliquer une méthode de ML.

### Cependant, les Kmeans ne renvoies pas de probabilité associée à la classe. Nous choisissons donc d'utiliser les Mixtures Gaussiennes, une généralisation de l'algorithme des Kmeans, mais qui leur associe une covariance. Cette covariance peut capturer les caractéristiques de chaque station, en fonction de leur historique d'intervention.
On pourra alors, pour chaque lieu d'incident, déterminer quelles station sont susceptibles d'intervenir. On pourra également vérifier si parmis les stations candidates, la station du jeu de donnée est bien présente.

In [ ]:
from sklearn.mixture import GaussianMixture as GM
def learn_test(a, b, subset):
    # le modèle marche mieux avec des morceau de segment, contrôle de la portion la plus performante à faire
    subset['middle_x'] = subset.o_x_utm + (a/b)*(subset.x_utm-subset.o_x_utm)
    subset['middle_y'] = subset.o_y_utm + (a/b)*(subset.y_utm-subset.o_y_utm)
    
    index = np.arange(subset.shape[0])
    subset = subset.set_index(index)
    np.random.shuffle(index)
    ratio_test = int(0.2*subset.shape[0])
    # on calcul le poids de chaque station (nombre d'intervention par station / nombre total d'intervention)
    w = np.array(subset.loc[index[:-ratio_test], ['DeployedFromStation_Name']].value_counts().tolist()) / (subset.shape[0]-ratio_test)

    # on initialise la mixture gaussienne avec autant de centre que de station, initialisé sur les stations, pondéré par le taux d'intervention
    gm = GM(n_components=centers.shape[0], means_init=centers[:, :2], n_init=1, random_state=1337 )

    # fit et sub pour le test
    gm.fit(subset.loc[index[:-ratio_test], ['middle_x','middle_y']])
    print(a, b, " fit ok")
    y = subset.loc[index[-ratio_test:], ['DeployedFromStation_Name']]
    gm.means_=centers[:, :2].astype(float) #remets les stations aux centres des mixtures
    y_pred = gm.predict_proba(subset.loc[index[-ratio_test:], ['x_utm','y_utm']])
    y_pred = np.where(y_pred>0.0001)
    
    count_prob = []
    for i in np.unique(y_pred[0]):
        ind = y_pred[0]
        count_prob.append(y_pred[1][ind==i])
    print(a, b, " collecte ok")
    success = 0
    for i, c in enumerate(count_prob):
        if np.isin(y.iloc[i, 0], centers[c, -1]):
            success += 1/(len(count_prob))

    return success
results_gm = []
for a in range(11):
    results_gm.append(learn_test(a, 10, subset))


In [ ]:
plt.figure()
plt.plot(results_gm)
plt.show()

# Avec cette approche, on dépasse les 95% de succès. On observe également que le modèle est plus performant lorsqu'il est entraîné avec l'intégralité de la distance station-incident.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(results_km, color='b', label='Kmeans')
ax.plot(results_gm, color='r', label='GMM')
ax.set_ylim([0, 1])
ax.set_xticks(np.arange(11))
ax.set_xticklabels((np.arange(11)*0.1).round(2))
ax.set_xlabel('Ratio du segment station-incident', fontsize=20)
ax.set_ylabel('Ratio de réussite', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.legend(fontsize=20)
plt.tight_layout()

plt.show()

In [ ]:
# sample avec trajets

subset['middle_x'] = subset.o_x_utm + (10/10)*(subset.x_utm-subset.o_x_utm)
subset['middle_y'] = subset.o_y_utm + (10/10)*(subset.y_utm-subset.o_y_utm)
print(subset.shape[0])

# on calcul le poids de chaque station (nombre d'intervention par station / nombre total d'intervention)
w = np.array(subset.loc[:, ['DeployedFromStation_Name']].value_counts().tolist()) / (subset.shape[0])

# on initialise la mixture gaussienne avec autant de centre que de station, initialisé sur les stations, pondéré par le taux d'intervention
gm = GM(n_components=centers.shape[0], means_init =centers[:, :2], n_init=1, weights_init = w)
# fit et sub pour le test
gm.fit(subset.loc[:, ['middle_x','middle_y']])
gm.means_=centers[:, :2].astype(float) #remets les stations aux centres des mixtures
gmm_name = 'gmm_29_04_21'
np.save(gmm_name + '_weights', gm.weights_, allow_pickle=False)
np.save(gmm_name + '_means', gm.means_, allow_pickle=False)
np.save(gmm_name + '_covariances', gm.covariances_, allow_pickle=False)
print(gm.means_.shape)

In [ ]:
import numpy as np
import pandas as pd
import compute_path as cp
from sklearn import mixture
from bokeh.plotting import figure
from bokeh.io import  push_notebook,output_notebook, show
from bokeh.models import ColumnDataSource, LabelSet,Ellipse, Column
from bokeh.models import PreText, CustomJS,HTMLTemplateFormatter
from bokeh.models import TapTool, Div, DataTable, TableColumn, TextInput
from bokeh.tile_providers import get_provider
from bokeh.models.tools import HoverTool
from bokeh.layouts import row, layout, column, widgetbox
from streamlit_bokeh_events import streamlit_bokeh_events

import pickle as pickle 

station_coord = pd.read_csv('Station_coord.csv')
station_coord = station_coord.rename(columns={'long':'lon'})
subset = subset.rename(columns={'d_long':'lon', 'd_lat':'lat'})

def make_ellipses(p, gmm_name, station_coord):
    means = np.load(gmm_name + '_means.npy')
    covar = np.load(gmm_name + '_covariances.npy')
    gmm = mixture.GaussianMixture(n_components = len(means), covariance_type='full')
    gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
    gmm.weights_ = np.load(gmm_name + '_weights.npy')
    gmm.means_ = means
    gmm.covariances_ = covar
    vs, angles = np.zeros((len(gmm.covariances_),2)), np.zeros((len(gmm.covariances_)))
    for n in range(len(gmm.covariances_)):
        covariances = gmm.covariances_[n][:2, :2]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        angles[n] = angle
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        vs[n, :] = v

    stations = pd.DataFrame(gmm.means_, columns=['x','y'])
    stations['width'] = vs[:, 0]
    stations['height'] = vs[:, 1]
    stations['angle'] = angles+180
    stations['names'] = subset.DeployedFromStation_Name.unique()
    source = ColumnDataSource(stations)
    ellipses = p.ellipse(x='x', y='y', width='width', height='height', angle='angle', fill_color="#cab2d6", fill_alpha=0.2, source=source)
    return ellipses, station, source

tuile = get_provider('CARTODBPOSITRON_RETINA')
tuile2 = get_provider('CARTODBPOSITRON_RETINA')
lon, lat = cp.wgs84_to_web_mercator(station_coord.lat, station_coord.lon)
station_coord['x'] = lon
station_coord['y'] = lat
source = ColumnDataSource(station_coord)
# On récupère les valeurs mini/maxi des coordonnées,
# on rajoute 10km pour avoir une carte centrée
xutm_min, xutm_max, yutm_min, yutm_max = station_coord.x.min()-10000, station_coord.x.max()+10000, \
                                         station_coord.y.min()-10000, station_coord.y.max()+10000

x_range, y_range, x_axis_type, y_axis_type = (xutm_min, xutm_max), (yutm_min, yutm_max), 'mercator', 'mercator'

p = figure(x_range=x_range, y_range=y_range, x_axis_type=x_axis_type, y_axis_type=y_axis_type,
           plot_width=1200, plot_height=1200)
p.add_tile(tuile)

station = p.circle(x='x',y='y', size=5, fill_color='blue', source=source)

ellipses, station_df, source_ell = make_ellipses(p, 'gmm_29_04_21', station_coord)
p.xaxis.major_label_text_font_size = '25px'
p.xaxis.axis_label = "Lon"
p.xaxis.axis_label_text_font_size = '25px'

p.yaxis.major_label_text_font_size = '25px'
p.yaxis.axis_label = "Lat"
p.yaxis.axis_label_text_font_size = '25px'
show(p)